In [1]:
import copy
import random
import sys
import time
import plotly.figure_factory as ff
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calculateMakespan(times, machines, config, n):
    time_table = []
    times = copy.deepcopy(times)
    machines = copy.deepcopy(machines)
    mn = len(machines[0])
    for i in range(mn):
        time_table.append([])

    current_times = [0]*n
    total_time = 0
    for j in config:
        job = j%n
        current_machine = machines[job].pop(0)
        current_time = current_times[job]
        machine_usage = time_table[current_machine]
        usage_time = times[job].pop(0)
        current_time, total_time = fillTimeSlot(machine_usage, current_time, usage_time, job, total_time)

        current_times[job] = current_time

    return total_time, time_table


def fillTimeSlot(machine_usage, current_time, usage_time, job, total_time):
    if len(machine_usage) > 0:
        prev = 0
        slot = None
        for used_slots in machine_usage:
            start = used_slots[0]
            end = used_slots[1]

            if start < current_time and current_time < end:
                current_time = end

            if prev == 0 and start > current_time + usage_time:
                slot = [current_time, usage_time + current_time, job]
                break
            elif start - prev >= usage_time and current_time <= prev:
                slot = [current_time, current_time + usage_time, job]
                break

            prev = end
            if end > current_time:
                current_time = end

        if slot == None:
            slot = [current_time, current_time + usage_time, job]

        current_time = slot[1]
        machine_usage.append(slot)
        machine_usage.sort(key=lambda x: x[1])

        if slot[1] > total_time:
            total_time = slot[1]

    else: 
        machine_usage.append([current_time, usage_time + current_time, job])
        current_time += usage_time

    return current_time, total_time

In [3]:
def readFilePairs(filepath):
    times_done = False
    times = []
    machines = []

    with open(filepath) as fp:
        line = fp.readline()
        n, mn = line.strip().split(' ')
        line = fp.readline()
        # print(line)

        while line:
            parse_line = ' '.join(line.split())
            raw_line = parse_line.strip().split(' ')
            curr = []
            i = 0
            machine = []
            time = []
            while i < len(raw_line):
                m, t = raw_line[i], raw_line[i + 1]
                machine.append(int(m))
                time.append(int(t))
                i += 2

            times.append(time)
            machines.append(machine)
            line = fp.readline()

    return times, machines, int(n)

def readSolution(filepath):
    machines = []

    with open(filepath) as fp:

        line = fp.readline()
        while line:
            raw_line = line.strip().split(' ')
            curr = []
            for char in raw_line:
                if len(char) > 0:
                    curr.append(int(char))
            machines.append(curr)
            line = fp.readline()
    sequence = []
    for i in range(len(machines[0])):
        for j in range(len(machines)):
            sequence.append(machines[j][i])
    return sequence

def swap_rnd(config):
    id1 = random.choice(range(len(config)))
    id2 = random.choice(range(len(config)))
    tmp = config[id1]
    config[id1] = config[id2]
    config[id2] = tmp
    return config

def fromPermutation(permutation, n):
    return list(map(lambda  x: x%n, permutation))
    

def testPermutation(permutation, times, machines, n):
    best_result, table = calculateMakespan(times, machines, permutation, n)
    print("SEQUENCE")
    print(permutation)
    print("RESULT:")
    print(best_result)
    job_sequence = []
    print("TABLE:")
    i = 0
    for row in table:
        print("M%s: %s" %(i, row))
        for slot in row:
            job_sequence.append(slot[2])
        i += 1
    print(job_sequence)

def printTable(table):
    i = 0
    print("TABLE: ")
    for row in table:
        print("M%s: %s" %(i, row))
        i += 1

In [4]:
def removeFromList(parent, list):
    seen = set()
    seen_add = seen.add
    return [x for x in parent if not (x in list or seen_add(x))]


def replaceWithRandomPopulation(population, q, n, mn):
    for i in range(q):
        population.pop()
    for i in range(q):
        addRandomIndividual(population, n, mn)

def checkDiversity(population, diff, n, mn):
    if diff < 0.03:
        replaceWithRandomPopulation(population, int(n/3), n, mn)
    if diff < 0.05:
        replaceWithRandomPopulation(population, int(n/5), n, mn)
    elif diff < 0.1:
        replaceWithRandomPopulation(population, int(n/10), n, mn)

def getFitness(population):
    prev = population[0][1]
    total = 0
    diffPercentage = 0.0
    for ind in population:
        curr = ind[1]
        total += curr
        diffPercentage += (curr/float(prev)) - 1
        prev = curr

    return total, diffPercentage

#Each individual is compose by a permutation(list from 0 to the job_number*machine_number)
#And a second parameter that is filled with the result of the makespan for the permutation
#We keep track of the result to not calculate multiple times the same result unnecesarily
#Is important to remove that number every time the permutation change
def addRandomIndividual(population, n, mn):
    ind = list(range(n*mn))
    random.shuffle(ind)
    population.append([ind, None])


#We generate the number of population
def generate_population(number, n, mn):
        population = []
        for i in range(number):
            addRandomIndividual(population, n, mn)
        return population

#During the crossover we select gens from the father from the start to the end index defined, we remove those from the mother
#Then we add them to the resultant in the same order that it was in the father origininally
def crossover(father, mother, start_index, end_index):
    father_gen = father[0][start_index:end_index]
    fetus = removeFromList(mother[0], father_gen)
    result = []
    result.extend(fetus[:start_index])
    result.extend(father_gen)
    result.extend(fetus[start_index:])
    return [result, None]



#mutate one member of the poupulation randomly excluding the first one(best individual)
#We just change the order of the permutation by one
def mutation(population, mutation_rate):
    if(random.random() > mutation_rate):
        candidate = random.choice(population[1:])
        swap_rnd(candidate[0])
        candidate[1] = None

def evolve(population, mutation_rate):
    #Important: the population should be sorted before evolve

    #We delete the worst individual of the population
    population.pop()

    #we choose a mother and father for the new individual
    father = random.choice(population)
    mother = random.choice(population)
    while(mother == father):
        mother = random.choice(population)
    
    indexes = range(len(father[0]))

    #we select wich part of the father will go to the mother
    start_index = random.choice(indexes)
    end_index = random.choice(indexes[start_index:])

    #we generate the baby with the crossover
    baby = crossover(father, mother, start_index, end_index)

    #we add the new member to the population
    population.append(baby)

    #we trigger the mutation for one of the population, depending on the mutation rate
    mutation(population, mutation_rate)
    return population

In [5]:
def plotResult(table, maxValue):
    df = []
    mn = 0
    colors = []
    for row in table:
        mn += 1
        row.sort(key=lambda x: x[2])
        for slot in row:
            start_time=str(datetime.timedelta(seconds=slot[0]))
            end_time=str(datetime.timedelta(seconds=slot[1]))
            today = datetime.date.today()
            entry = dict(
                Task='Machine-{0}'.format(mn-1), 
                Start="{0} {1}".format(today, start_time), 
                Finish="{0} {1}".format(today, end_time),
                duration=slot[1] - slot[0],
                Resource='Job {0}'.format(slot[2] + 1)
                )
            df.append(entry)

            #Generate random colors
            if(len(colors) < len(row)):
                a = min(255 - ( slot[2] * 20 ), 255)
                b = min(slot[2] * 10, 255)
                c = min(255, int(random.random() * 255))
                colors.append("rgb({0}, {1}, {2})".format(a, b, c))

    #In order to see the line ordered by integers and not by dates we need to generate the dateticks manually
    #we create 11 linespaced numbers between 0 and the maximum value
    num_tick_labels = np.linspace(start = 0, stop = maxValue, num = 11, dtype = int)
    date_ticks = ["{0} {1}".format(today, str(datetime.timedelta(seconds=int(x)))) for x in num_tick_labels]
    fig = ff.create_gantt(df,colors=colors, index_col='Resource', group_tasks=True, show_colorbar=True, showgrid_x=True, title='Job shop Schedule')
    fig.layout.xaxis.update({
        'tickvals' : date_ticks,
        'ticktext' : num_tick_labels
        })
    fig.show()

In [6]:
def printProgress(bestValue, iterations, timeElapsed):
    sys.stdout.write("\rIterations: {0} | Best result found {1} | Time elapsed: {2}s".format(iterations, bestValue, int(timeElapsed)))
    sys.stdout.flush()

def genetic(times, machines, n, population_number, iterations, rate, target):
    machine_number = len(machines[0])
    start_time = time.time()

    def sortAndGetBestIndividual(population):
        best_individual = None
        best_result = None
        for individual in population:
            result = None
            if not individual[1]: 
                result, table = calculateMakespan(times, machines, individual[0], n)
                individual[1] = result
            else: 
                result = individual[1]

            if not best_result or result < best_result:
                best_result = result
                best_individual = individual

        population.sort(key=lambda x: x[1])
        return best_individual, best_result

    population = generate_population(population_number, n, machine_number)
    global_best_ind, global_best = sortAndGetBestIndividual(population)
    
    ##if we don't define a target we set the number of iterations we want 
    if not target:
        for i in range(iterations):
            population = evolve(population, rate)
            best_ind, best_result = sortAndGetBestIndividual(population)
            total_fitness, diffPercentage = getFitness(population)

            if(not global_best or best_result < global_best):
                global_best = best_result
                global_best_ind = copy.deepcopy(best_ind)

            printProgress(best_result, i, time.time() - start_time)
            checkDiversity(population, diffPercentage, n, machine_number)
    else:
        #If we define a target we iterate until the best result reach that target
        i = 0
        while(target < global_best):
            i += 1
            #in every iteration: 
            #We evolve the population
            population = evolve(population, rate)
            #We find the best individual 
            best_ind, best_result = sortAndGetBestIndividual(population)
            #We calculate the diversity % between the population and the total_fitness(sum of all the results)
            total_fitness, diffPercentage = getFitness(population)

            #if the result found is better than the global found we update the global
            if(not global_best or best_result < global_best):
                global_best = best_result
                global_best_ind = copy.deepcopy(best_ind)
            #We print the progress so far and the time elapsed
            printProgress(best_result, i, time.time() - start_time)
            #We check the diversity, in case the diversity percentage is very low we delete a number of the population and we add randome members
            checkDiversity(population, diffPercentage, n, machine_number)

    
    best_result, best_table = calculateMakespan(times, machines, global_best_ind[0], n)           
    print("\nOVERALL RESULT")
    print("RESULT: %s" %best_result)                 
    print('the elapsed time:%ss'% (int(time.time() - start_time)))
    print("Permutation: ")
    print(fromPermutation(global_best_ind[0], n))
    printTable(best_table)
    plotResult(best_table, best_result)


In [7]:
target = None

population_size = 10
mutation_rate = 0.1

# population_size=int(input('Please input the size of population (default: 10): ') or 10)
# mutation_rate=float(input('Please input the size of Mutation Rate (default 0.1): ') or 0.1)
iterations=int(input('Please input number of iteration (default 1000): ') or 1000 )

times, machines, n = readFilePairs("/content/20_5_la11.txt")
genetic(times, machines, n, population_size, iterations, mutation_rate, target)

Please input number of iteration (default 1000): 
Iterations: 999 | Best result found 1222 | Time elapsed: 5s
OVERALL RESULT
RESULT: 1222
the elapsed time:5s
Permutation: 
[5, 0, 17, 10, 11, 13, 13, 17, 11, 13, 8, 5, 4, 7, 0, 12, 12, 17, 11, 9, 1, 15, 19, 7, 16, 6, 2, 2, 11, 6, 12, 8, 7, 11, 18, 9, 14, 3, 10, 8, 4, 0, 4, 5, 1, 16, 1, 15, 19, 6, 12, 13, 4, 2, 7, 19, 18, 16, 15, 19, 5, 15, 3, 14, 18, 8, 13, 7, 1, 3, 17, 18, 10, 17, 9, 14, 10, 14, 4, 15, 3, 5, 6, 16, 8, 16, 1, 9, 2, 14, 10, 0, 9, 0, 19, 12, 18, 6, 3, 2]
TABLE: 
M0: [[0, 95, 10], [95, 128, 17], [128, 211, 4], [211, 272, 11], [272, 368, 9], [368, 389, 1], [389, 482, 6], [482, 494, 2], [494, 538, 8], [538, 591, 0], [591, 683, 5], [683, 703, 15], [703, 749, 12], [749, 777, 13], [777, 846, 18], [846, 933, 14], [933, 993, 7], [993, 1048, 3], [1048, 1126, 16], [1126, 1222, 19]]
M1: [[0, 89, 17], [89, 141, 13], [141, 162, 0], [162, 253, 12], [253, 307, 15], [307, 316, 11], [316, 365, 8], [365, 406, 7], [406, 481, 9], [481, 488, 1